<a href="https://colab.research.google.com/github/AlineCrawf/datasciencecoursera/blob/master/RNN_Example_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Импорт модулей

In [ ]:
from __future__ import print_function
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN
from keras.models import Sequential
from keras.utils.vis_utils import plot_model
import numpy as np

Предварительная обработка текста из "Алисы в Стране чудес"

In [ ]:
!wget http://www.gutenberg.org/files/11/11-0.txt

--2020-12-24 17:08:34--  http://www.gutenberg.org/files/11/11-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174693 (171K) [text/plain]
Saving to: ‘11-0.txt’

11-0.txt            100%[===================>] 170.60K   416KB/s    in 0.4s    

2020-12-24 17:08:35 (416 KB/s) - ‘11-0.txt’ saved [174693/174693]



In [ ]:
fin = open("11-0.txt", 'rb') #Вставить сюда название файла, который загрузила команда выше
lines = []
for line in fin:
  line = line.strip().lower()
  line = line.decode("ascii", "ignore")
  if len(line) == 0:
    continue
  lines.append(line)
fin.close()
text = " ".join(lines)

Создаём таблицы соответствия символов и их индексов

In [ ]:
chars = set([c for c in text])
nb_chars = len(chars)
char2index = dict((c, i) for i, c in enumerate(chars))
index2char = dict((i, c) for i, c in enumerate(chars))
print(chars)

{'(', '9', '1', 'e', '_', '?', 'v', '!', 'i', 'x', ':', '8', '0', "'", '6', '"', 'r', '4', 'z', 'd', '$', 'k', 'u', 'n', 'w', 'y', ')', '5', '[', ',', ']', 'a', 'g', '%', ';', '/', 'l', 't', 's', '7', ' ', 'f', 'm', 'b', '@', 'q', '.', '2', '#', 'p', 'c', 'o', '3', '-', 'j', 'h', '*'}


Создание входных строк и меток. Проходим по тексту с шагом в 1 символ и выделяем отрезок длиной 10 сиволов. Символ после отрезка является меткой

In [ ]:
SEQLEN = 10
STEP = 1
 
input_chars = []
label_chars = []
for i in range(0, len(text) - SEQLEN, STEP):
  input_chars.append(text[i:i + SEQLEN])
  label_chars.append(text[i + SEQLEN])

Векторизация входных строк и меток

In [ ]:
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
  for j, ch in enumerate(input_char):
    X[i, j, char2index[ch]] = 1
  y[i, char2index[label_chars[i]]] = 1

Размерность выхода РНС - 128 (определяется в ходе экспериментов). Будем получать на выходе 1 символ, а не последовательность, поэтому return_sequences=False. Входные данные имеют форму матрицы SQLEN*nb_chars. Чтобы повысить качество работы библиотеки TensorFlow, зададим unroll=True. РНС соединяется с полносвязным (плотным) слоем, в котором nb_char нейронов, которые выдают оценки появления каждого символа из словаря. Функция активации - softmax, которая нормаирует оценки, преобразуя их в вероятности. Функция потерь - категориальная перекрестная энтропия (УРА, Я кажется поняла что это значит!) 

In [ ]:
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 400
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100
 
model = Sequential()
model.add(SimpleRNN(HIDDEN_SIZE, return_sequences=False, 
                    input_shape=(SEQLEN, nb_chars), unroll=True))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

Поскольку в данном случае у нас нет помеченных данных для тестирования, выполняем один период обучения (NUM_EPOCHS_PER_ITERATION = 1), а затем тестируем модель. Так происходит на протяжении 25 итераций, следовательно, по существу мы выполним 25 периодов обучения и тестируем модель прсле каждого периода. 

Тестирование производится так: модель порождает символ по заданным входным данным, затем первый символ входной строки отбрасывается, в конец дописывается предсказанный на предыдущем прогоне символ и у модели запрашивается следующее предсказание. Так повторяется 100 раз, после чего получившаяся строка печатается. Эта строка и является индикатором качества модели. 

In [ ]:
for iteration in range(NUM_ITERATIONS):
  print("=" * 50)
  print("Iteration #: %d" % (iteration))
  model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)
 
  test_idx = np.random.randint(len(input_chars))
  test_chars = input_chars[test_idx]
  print("Generating from seed: %s" % (test_chars))
  print(test_chars, end="")
  for i in range(NUM_PREDS_PER_EPOCH):
    Xtest = np.zeros((1, SEQLEN, nb_chars))
    for i, ch in enumerate(test_chars):
      Xtest[0, i, char2index[ch]] = 1
    pred = model.predict(Xtest, verbose=0)[0]
    ypred = index2char[np.argmax(pred)]
    print(ypred, end="")
    #сдвинуться вперед на test_chars + ypred
    test_chars = test_chars[1:] + ypred
print()

Iteration #: 0
1249/1249 [==============================] - 9s 7ms/step - loss: 2.5852
Generating from seed: le on it, 
le on it, and the wast of and the said the wast of and the said the wast of and the said the wast of and the s==================================================
Iteration #: 1
1249/1249 [==============================] - 8s 7ms/step - loss: 2.0604
Generating from seed:  go round 
 go round of the master and and all the was at and all the was at and all the was at and all the was at and al==================================================
Iteration #: 2
1249/1249 [==============================] - 8s 7ms/step - loss: 1.9571
Generating from seed:  changed s
 changed so the saided and and alice to the rooken whe said the master alice to the rooken whe said the master==================================================
Iteration #: 3
1249/1249 [==============================] - 8s 7ms/step - loss: 1.8769
Generating from seed: ted by the
ted by the parse for the poor and the

KeyboardInterrupt: ignored